## 3.1 Simulating Data

In [0]:
import pandas as pd
store_sales = pd.read_csv('http://bit.ly/PMR-ch3')

### 3.1.1 Store Data: Setting the Structure

In [0]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Constants
N_STORES = 20
N_WEEKS = 104

# create a data frame of initially missing values to hold the data
columns = ('store_num', 'year', 'week', 'p1_sales', 'p2_sales',
           'p1_price', 'p2_price', 'p1_promo', 'p2_promo', 'country')
n_rows = N_STORES * N_WEEKS
store_sales = pd.DataFrame(np.empty(shape=(n_rows, 10)),
                           columns=columns)

In [0]:
store_sales.shape

In [0]:
store_sales.head()

In [0]:
store_numbers = range(101, 101 + N_STORES)
list(store_numbers)

In [0]:
store_country = dict(zip(store_numbers,
                         ['USA', 'USA', 'USA', 'DEU', 'DEU', 'DEU',
                          'DEU', 'DEU', 'GBR', 'GBR', 'GBR', 'BRA',
                          'BRA', 'JPN', 'JPN', 'JPN', 'JPN', 'AUS',
                          'CHN', 'CHN']))
store_country

In [0]:
i = 0
for store_num in store_numbers:
  for year in [1, 2]:
    for week in range(1, 53):
      store_sales.loc[i, 'store_num'] = store_num
      store_sales.loc[i, 'year'] = year
      store_sales.loc[i, 'week'] = week
      store_sales.loc[i, 'country'] = store_country[store_num]
      i += 1

In [0]:
store_sales.head()

In [0]:
store_sales.dtypes

In [0]:
type(store_sales.country[0])

In [0]:
store_sales.country = store_sales.country.astype(
    pd.CategoricalDtype())
store_sales.store_num = store_sales.store_num.astype(
    pd.CategoricalDtype())
print(store_sales.store_num.head())
print(store_sales.country.head())

In [0]:
store_sales.dtypes

In [0]:
# Not shown
store_sales.head(60) # 60 rows can be displayed without truncation;
store_sales.tail(60) # make sure end looks OK too;
store_sales.sample(60) # inspecting a random sample is also helpful;


### 3.1.2 Store Data: Simulating Data Points

In [0]:
np.random.seed(37204)

In [0]:
np.random.binomial(n=1, p=0.1, size=n_rows).shape

In [0]:
store_sales.shape

In [0]:
# 10% promoted
store_sales.p1_promo = np.random.binomial(n=1, p=0.1, size=n_rows)
# 15% promoted
store_sales.p2_promo = np.random.binomial(n=1, p=0.15, size=n_rows)
store_sales.head(10) # how does it look so far?

In [0]:
store_sales.p1_promo.value_counts()

In [0]:
store_sales.p1_price = np.random.choice([2.19, 2.29, 2.49, 2.79, 
                                         2.99],
                                     size=n_rows)
store_sales.p2_price = np.random.choice([2.29, 2.49, 2.59, 2.99,
                                         3.19],
                                     size=n_rows)
store_sales.sample(5) # now how does it look?

In [0]:
# sales data, using poisson (counts) distribution
# np.random.poisson()
# first, the default sales in the absence of promotion
sales_p1 = np.random.poisson(lam=120, size=n_rows)
sales_p2 = np.random.poisson(lam=100, size=n_rows)

In [0]:
# scale sales according to the ratio of log(price)
log_p1_price = np.log(store_sales.p1_price)
log_p2_price = np.log(store_sales.p2_price)

sales_p1 = sales_p1 * log_p2_price/log_p1_price
sales_p2 = sales_p2 * log_p1_price/log_p2_price

In [0]:
# final sales get a 30% or 40% lift when promoted
store_sales.p1_sales = np.floor(sales_p1 *
                                (1 + store_sales.p1_promo * 0.3))
store_sales.p2_sales = np.floor(sales_p2 *
                                (1 + store_sales.p2_promo * 0.4))
store_sales.sample(10)

## 3.2 Functions to Summarize a Variable

### 3.2.1 Language brief: groupby()

In [0]:
store_sales.groupby('store_num')

In [0]:
store_sales.groupby('store_num').p1_sales.mean()

In [0]:
store_sales.groupby(['store_num', 'year']).p1_sales.mean()

In [0]:
store_sales.groupby(['store_num', 'year']).p1_sales.mean().unstack()

In [0]:
p1_sales_by_country = store_sales.groupby(['country']).p1_sales.sum()
p1_sales_by_country

### 3.2.1 Discrete Variables

In [0]:
store_sales.p1_price.value_counts()

In [0]:
p1_table_0 = store_sales.p1_price.value_counts()
p1_table_0

In [0]:
type(p1_table_0)

In [0]:
p1_table_0.plot.bar()

In [0]:
pd.crosstab(store_sales.p1_promo, store_sales.p1_price)

In [0]:
store_sales.groupby('p1_promo').p1_price.value_counts().unstack()

In [0]:
store_sales.groupby(['p1_promo', 'country']).p1_price.mean().unstack()

In [0]:
p1_table_1 = store_sales.groupby('p1_promo').p1_price.value_counts()
p1_table_1 = p1_table_1.unstack()
p1_table_1.div(p1_table_0)

### 3.2.2 Continuous Variables

In [0]:
store_sales.p2_sales.min()

In [0]:
store_sales.p1_sales.max()

In [0]:
store_sales.p1_promo.mean()

In [0]:
store_sales.p2_sales.median()

In [0]:
store_sales.p1_sales.var()

In [0]:
store_sales.p1_sales.std()

In [0]:
store_sales.p1_sales.mad()

In [0]:
store_sales.p1_sales.quantile(q=[0.25, 0.5, 0.75])

In [0]:
store_sales.p1_sales.quantile(q=[0.05, 0.95])

In [0]:
store_sales.p1_sales.quantile(q=np.arange(0, 1.1, 0.1))

In [0]:
def iqr(x):
  '''Return the interquartile range of the input numpy array'''
  return x.quantile(0.75) - x.quantile(0.25)
iqr(store_sales.p1_sales)

In [0]:
pd.DataFrame([[store_sales.p1_sales.median(),
               store_sales.p2_sales.median()],
              [iqr(store_sales.p1_sales), iqr(store_sales.p2_sales)]],
             index=['Median sales', 'IQR'],
             columns=['p1_sales', 'p2_sales'])

## 3.3 Summarizing Data Frames

In [0]:
store_sales.describe()

In [0]:
store_sales.p1_price.describe()

### 3.3.4 apply()

In [0]:
store_sales.iloc[:, 3:9].mean()

In [0]:
store_sales.iloc[:, 3:9].mean(axis=1).head()

In [0]:
#store_sales.iloc[:, 3:9].iqr()

In [0]:
store_sales.iloc[:, 3:9].apply(iqr)

In [0]:
store_sales.iloc[:, 3:9].apply(iqr, axis=1).head()

In [0]:
store_sales.iloc[:, 3:9].apply(lambda x: x.mean() - x.median())

In [0]:
store_sales.p1_sales.apply(lambda x: 'high' if x > 130 else 'low')[:5]

In [0]:
pd.DataFrame([store_sales[['p1_sales', 'p2_sales']].median(),
              store_sales[['p1_sales', 'p2_sales']].apply(iqr)],
             index=['Median sales', 'IQR'])

## 3.4 Single Variable Visualization

### 3.4.1 Histograms

In [0]:
store_sales.p1_sales.hist()

In [0]:
import matplotlib.pyplot as plt
store_sales.p1_sales.hist()
plt.title('Product 1 weekly sales frequencies, All stores')
plt.xlabel('Product 1 sales (units)')
plt.ylabel('Count')

In [0]:
plt.rcParams.update({'font.size': 12})

In [0]:
store_sales.p1_sales.hist(bins=30,
                          edgecolor='k',
                          facecolor='lightblue')
plt.title('Product 1 weekly sales frequencies, All stores')
plt.xlabel('Product 1 sales (units)')
plt.ylabel('Count')
plt.grid(False)
plt.box(False)

In [0]:
store_sales.p1_sales.hist(bins=30,
                          edgecolor='k',
                          facecolor='lightblue',
                          density=True)
plt.title('Product 1 weekly sales frequencies, All stores')
plt.xlabel('Product 1 sales (units)')
plt.ylabel('Relative frequency')
plt.xticks(range(60, 300, 20))
plt.grid(False)
plt.box(False)

In [0]:
store_sales.p1_sales.hist(bins=30,
                          edgecolor='k',
                          facecolor='lightblue',
                          density=True)
store_sales.p1_sales.plot.density(color='red')
plt.title('Product 1 weekly sales frequencies, All stores')
plt.xlabel('Product 1 sales (units)')
plt.ylabel('Relative frequency')
plt.xticks(range(60, 300, 20))
plt.xlim((60, 290))
plt.box(False)

### Box plots

In [0]:
p = store_sales.p2_sales.plot.box(vert=False, sym='k.')
plt.title('Weekly sales of P2, All stores')
plt.xlabel('Weekly sales')
p.set_facecolor('w')

In [0]:
store_sales.boxplot(column='p2_sales', by='store_num', vert=False,
                    sym='k.')
plt.suptitle('')
plt.title('Weekly sales of p2 by store')
plt.xlabel('Weekly unit sales')
plt.ylabel('Store')
plt.box(False)

In [0]:
store_sales.boxplot(column='p2_sales', by='p2_promo', vert=False,
                    sym='k.')
plt.suptitle('')
plt.title('Weekly sales of p2 with and without promotion')
plt.xlabel('Weekly unit sales')
plt.ylabel('P2 promo in store?')
plt.yticks([1, 2], ['No', 'Yes'])
plt.box(False)

### 3.4.3 QQ Plot to check normality

In [0]:
from scipy import stats
plt.figure(figsize=(7,7))
stats.probplot(store_sales.p1_sales, dist='norm', plot=plt)

In [0]:
plt.figure(figsize=(7,7))
stats.probplot(np.log(store_sales.p1_sales), dist='norm', plot=plt)

### 3.4.4 Cumulative distribution

In [0]:
from statsmodels.distributions.empirical_distribution import ECDF
e = ECDF(store_sales.p1_sales)
plt.plot(e.x, e.y)
plt.title('Cumulative distribution of p1 weekly sales')
plt.ylabel('Cumulative proportion')
plt.plot([60, 270], [0.9, 0.9], 'k--', alpha=0.5)
plt.plot([store_sales.p1_sales.quantile(.9),
          store_sales.p1_sales.quantile(.9)],
         [0, 1], 'k--', alpha=0.5)
plt.box(False)

In [0]:
ecdf_x = store_sales.p1_sales.sort_values()
ecdf_y = np.arange(0, 1, 1/len(store_sales.p1_sales))
plt.subplot(2,1,2)
plt.plot(ecdf_x, ecdf_y)
plt.xlabel('P1 weekly sales, all stores')
plt.ylabel('Cumulative proportion')
plt.plot([60, 270], [0.9, 0.9], 'k--', alpha=0.5)
plt.plot([store_sales.p1_sales.quantile(.9),
          store_sales.p1_sales.quantile(.9)],
         [0, 1], 'k--', alpha=0.5)
plt.box(False)

### 3.4.6 Maps

In [0]:
!apt-get -qq install libproj-dev proj-data  proj-bin libgeos-dev 
!pip install cartopy

In [0]:
from cartopy.io import shapereader
from cartopy import crs

plt.figure(figsize=(16,6))
ax = plt.axes(projection=crs.PlateCarree())

shpfile = shapereader.natural_earth(resolution='110m',
                                  category='cultural',
                                  name='admin_0_countries')
reader = shapereader.Reader(shpfile)
countries = reader.records()
max_sales = p1_sales_by_country.max()
for country in countries:
  country_name = country.attributes['ADM0_A3']
  if country_name in p1_sales_by_country:
    ax.add_geometries(country.geometry, crs.PlateCarree(),
      facecolor=plt.cm.Greens(p1_sales_by_country[country_name]
                                 /max_sales),
      edgecolor='k')
  else:
    ax.add_geometries(country.geometry, crs.PlateCarree(),
      facecolor='w',
      edgecolor='k')